## Dependencies

In [ ]:
# ensemble_boxes 1.0.7
!pip install ../input/sartorius-cell-instance-segmentation-dataset/packages/packages/ensemble_boxes-1.0.7-py3-none-any.whl
# detectron2 0.5
!pip install ../input/sartorius-cell-instance-segmentation-dataset/packages/packages/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar --no-index --find-links ../input/sartorius-cell-instance-segmentation-dataset/packages/packages 
!pip install ../input/sartorius-cell-instance-segmentation-dataset/packages/packages/fvcore-0.1.5.post20211019/fvcore-0.1.5.post20211019 --no-index --find-links ../input/sartorius-cell-instance-segmentation-dataset/packages/packages 
!pip install ../input/sartorius-cell-instance-segmentation-dataset/packages/packages/antlr4-python3-runtime-4.8/antlr4-python3-runtime-4.8 --no-index --find-links ../input/sartorius-cell-instance-segmentation-dataset/packages/packages 
!pip install ../input/sartorius-cell-instance-segmentation-dataset/packages/packages/detectron2-0.5/detectron2 --no-index --find-links ../input/sartorius-cell-instance-segmentation-dataset/packages/packages
# cellpose 0.7.2
!pip install --no-index ../input/cellposeoffline/cellpose-0.7.2-py3-none-any.whl --find-links=../input/cellposeoffline

In [ ]:
import yaml
import os
import json
from glob import glob
from tqdm.notebook import tqdm
from typing import Union
from collections import Counter
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import cv2
from scipy.stats import mode
import matplotlib.pyplot as plt
from numba import jit
import torch
from fastai.vision.all import *
import detectron2
import detectron2.layers
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
import pycocotools.mask as mask_util
import networkx as nx

In [ ]:
RAW_DATASET_PATH = '../input/sartorius-cell-instance-segmentation'
DETECTRON_MODELS_PATH = '../input/sartorius-cell-instance-segmentation-dataset'
CLASSIFIER_PATH = '../input/sartorius-fastai-classifier'
CELLPOSE_MODELS_PATH = '../input/sartoriuscellposemodels'

In [ ]:
df_train = pd.read_csv(f'{RAW_DATASET_PATH}/train.csv')
df_test = pd.read_csv(f'{RAW_DATASET_PATH}/sample_submission.csv')

print(f'Training Set Shape: {df_train.shape} - {df_train["id"].nunique()} Images - Memory Usage: {df_train.memory_usage().sum() / 1024 ** 2:.2f} MB')
print(f'Test Set Shape: {df_test.shape} - {df_test["id"].nunique()} Images - Memory Usage: {df_test.memory_usage().sum() / 1024 ** 2:.2f} MB')

## Annotation Utilities

In [ ]:
def decode_rle_mask(rle_mask, shape, fill_holes=False, is_coco_encoded=False):

    """
    Decode run-length encoded mask string into 2d binary mask array

    Parameters
    ----------
    rle_mask (str): Run-length encoded mask string
    shape (tuple): Height and width of the mask
    fill_holes (bool): Whether to fill holes in masks or not
    is_coco_encoded (bool): Whether the mask is encoded with pycocotools or not

    Returns
    -------
    mask [numpy.ndarray of shape (height, width)]: Decoded 2d mask
    """

    if is_coco_encoded:
        # Decoding RLE encoded mask of string
        mask = np.uint8(mask_utils.decode({'size': shape, 'counts': rle_mask}))
    else:
        # Decoding RLE encoded mask of integers
        rle_mask = rle_mask.split()
        starts, lengths = [np.asarray(x, dtype=int) for x in (rle_mask[0:][::2], rle_mask[1:][::2])]
        starts -= 1
        ends = starts + lengths

        mask = np.zeros((shape[0] * shape[1]), dtype=np.uint8)
        for start, end in zip(starts, ends):
            mask[start:end] = 1

        mask = mask.reshape(shape[0], shape[1])

    if fill_holes:
        mask = ndimage.binary_fill_holes(mask).astype(np.uint8)

    return mask


def encode_rle_mask(mask):

    """
    Encode 2d binary mask array into run-length encoded mask string

    Parameters
    ----------
    mask [numpy.ndarray of shape (height, width)]: 2d mask

    Returns
    -------
    rle_mask (str): Run-length encoded mask string
    """

    mask = mask.flatten()
    mask = np.concatenate([[0], mask, [0]])
    runs = np.where(mask[1:] != mask[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def binary_to_multi_object_mask(binary_masks):

    """
    Encode multiple 2d binary masks into a single 2d multi-object segmentation mask

    Parameters
    ----------
    binary_masks [numpy.ndarray of shape (n_objects, height, width)]: 2d binary masks

    Returns
    -------
    multi_object_mask [numpy.ndarray of shape (height, width)]: 2d multi-object mask
    """

    multi_object_mask = np.zeros((binary_masks.shape[1], binary_masks.shape[2]))
    for i, binary_mask in enumerate(binary_masks):
        non_zero_idx = binary_mask == 1
        multi_object_mask[non_zero_idx] = i + 1

    return multi_object_mask


def mask_to_bounding_box(mask):

    """
    Get bounding box from a binary mask

    Parameters
    ----------
    mask [numpy.ndarray of shape (height, width)]: 2d binary mask

    Returns
    -------
    bounding_box [list of shape (4)]: Bounding box of the object
    """

    non_zero_idx = np.where(mask == 1)
    bounding_box = [
        np.min(non_zero_idx[1]),
        np.min(non_zero_idx[0]),
        np.max(non_zero_idx[1]),
        np.max(non_zero_idx[0])
    ]

    return bounding_box


## Metric Utilities

In [ ]:
def precision_at(ious, threshold):

    """
    Get true positives, false positives, false negatives and precision score from given IoUs at given threshold

    Parameters
    ----------
    ious [numpy.ndarray of shape (ground_truth_objects, prediction_objects)]: Intersection over union between all ground-truths and predicted segmentation masks
    threshold (float): Threshold on which the hits are calculated

    Returns
    -------
    tp (int): Number of true positives in IoU hit matrix
    fp (int): Number of false positives in IoU hit matrix
    fn (int): Number of false negatives in IoU hit matrix
    precision (float): Precision score of IoU hit matrix (0.0 <= precision <= 1.0)
    """

    hits = ious > threshold
    true_positives = np.sum(hits, axis=1) == 1
    false_positives = np.sum(hits, axis=0) == 0
    false_negatives = np.sum(hits, axis=1) == 0
    tp, fp, fn = (
        np.sum(true_positives),
        np.sum(false_positives),
        np.sum(false_negatives),
    )
    precision = tp / (tp + fp + fn)
    return tp, fp, fn, precision


def get_average_precision(ground_truth_masks, prediction_masks, thresholds=(0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95), verbose=True):

    """
    Calculate mean Average Precision (mAP) between ground truth and prediction masks on given thresholds

    Parameters
    ----------
    ground_truth_masks [numpy.ndarray of shape (n_objects, height, width)]: Ground truth binary segmentation masks
    prediction_masks [numpy.ndarray of shape (m_objects, height, width)]: Prediction binary segmentation masks
    thresholds (tuple): Thresholds on which the hits are calculated
    verbose (bool): Verbosity flag

    Returns
    -------
    average_precision (float): Average precision score of IoU hit matrix (0.0 <= average_precision <= 1.0)
    """

    prediction_masks = [mask_util.encode(np.asarray(mask, order='F')) for mask in prediction_masks]
    ground_truth_masks = [mask_util.encode(np.asarray(mask, order='F')) for mask in ground_truth_masks]
    ious = mask_util.iou(prediction_masks, ground_truth_masks, [0] * len(ground_truth_masks))

    precisions = []
    for threshold in thresholds:
        tp, fp, fn, precision = precision_at(ious=ious, threshold=threshold)
        precisions.append(precision)
        if verbose:
            print(f'Precision: {precision:.6f} (TP: {tp} FP: {fp} FN: {fn}) at Threshold: {threshold:.2f}')

    average_precision = np.mean(precisions)
    if verbose:
        print(f'Image Average Precision: {average_precision:.6f}\n')

    return average_precision


## Visualization Utilities

In [ ]:
def visualize_image(image, masks, title, path=None):

    """
    Visualize image along with its segmentation masks

    Parameters
    ----------
    image [numpy.ndarray of shape (height, width)]: Grayscale image
    masks [numpy.ndarray of shape (n_objects, height, width)]: Segmentation masks
    title (str): Title of the plot
    path (str or None): Path of the output file (if path is None, plot is displayed with selected backend)
    """

    fig, ax = plt.subplots(figsize=(16, 16))
    ax.imshow(image, cmap='gray')

    if masks is not None:
        masks = np.stack(masks)
        mask = np.any(masks > 0, axis=0)
        ax.imshow(mask, alpha=0.4)

    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.tick_params(axis='x', labelsize=15, pad=10)
    ax.tick_params(axis='y', labelsize=15, pad=10)
    ax.set_title(title, size=20, pad=15)

    if path is None:
        plt.show()
    else:
        plt.savefig(path)
        plt.close(fig)


## Cell Type Classifier Inference

In [ ]:
from glob import glob

CLASS_NAMES = ['cort', 'astro', 'shsy5y']
FASTAI_CLF_LEARNER = f'{CLASSIFIER_PATH}/clf_resnet34.pkl'
FASTAI_CLF_LEARNERS = [f'{CLASSIFIER_PATH}/clf_resnet34_{fold}.pkl' for fold in range(5)]

df_test = pd.DataFrame(glob(f'{RAW_DATASET_PATH}/test/*'), columns=['image_path'])
df_test['id'] = df_test.image_path.map(lambda x: x.split('/')[-1].split('.')[0])
df_test.head()

In [ ]:
def load_RGBY_image(image_id):

    filename = f'{RAW_DATASET_PATH}/test/{image_id}.png'
    img = cv2.imread(filename)[..., 0]
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    img2 = clahe.apply(img)
    img3 = cv2.equalizeHist(img)
    stacked_images = np.stack([img, img2, img3], axis=-1)
    return stacked_images


In [ ]:
out_image_dir = '/kaggle/work/mmdet_test/'
!mkdir -p {out_image_dir}

for idx in tqdm(range(len(df_test))):
    image_id = df_test.iloc[idx]['id']
    img = load_RGBY_image(image_id)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    cv2.imwrite(f'{out_image_dir}/{image_id}.png', img)


In [ ]:
def get_test_classes(learner_path, test_folder, cpu=True):
    ids = [os.path.basename(fn).replace('.png', '') for fn in get_image_files(test_folder)]
    inference_learn = load_learner(
        learner_path,
        cpu=cpu,
    )
    test_dl = inference_learn.dls.test_dl(get_image_files(test_folder))
    preds, _ = inference_learn.get_preds(dl=test_dl)
    preds = preds.detach().cpu().numpy().argmax(1)
    return {_id: int(pred) for _id, pred in zip(ids, preds)}

# Single CLF model
id2class = get_test_classes(
    learner_path=f'{CLASSIFIER_PATH}/clf_resnet34.pkl',
    test_folder=out_image_dir,
    cpu=True
)

# Ensemble of CLF fold models
id2classes = [
    get_test_classes(
        learner_path=learner_fn,
        test_folder=out_image_dir,
        cpu=True
    ) for learner_fn in FASTAI_CLF_LEARNERS
]

# Sanity check visualization
f,axs = plt.subplots(1, 3, figsize=(12, 6))
for key, i in zip(id2class.keys(), range(3)):
    _cl = id2class[key]
    img_fn = os.path.join(out_image_dir, key + '.png')
    axs[i].imshow(cv2.imread(img_fn))
    axs[i].set_title(f'{key} - class {CLASS_NAMES[_cl]}')

In [ ]:
majority_id2classes = {}
for key in id2class.keys():
    preds = [d[key] for d in id2classes]
    counter = Counter(preds)
    majority_id2classes[key] = counter.most_common(1)[0][0]

# Save predictions to file
with open('id2class.json', 'w') as json_file:
    json.dump(majority_id2classes, json_file)
    
majority_id2classes

## Cellpose Inference

In [ ]:
%%writefile cellpose_inference.py

import numpy as np
from cellpose import models, io, plot
from pathlib import Path
import pandas as pd
import os
import json
import pickle
import cv2
from pycocotools import mask as maskUtils


def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    return img.reshape(shape)


def build_binary_mask(labels ,input_shape=(520,704)):
    """ """
    height, width = input_shape
    
    mask = np.zeros((height, width, 1))
    for label in labels:
        mask += rle_decode(label, shape=(height, width, 1))
        
    mask = mask.clip(0, 1)
    return mask.astype(np.uint8)


def ious_of_rles(rles):
    binary_masks = [build_binary_mask([rle]) for rle in rles]
    
    if len(binary_masks) == 0:
        return np.array([[0. for _ in range(len(binary_masks))]])
    
    enc_masks = [maskUtils.encode(np.asarray(p[:,:,0], order='F')) for p in binary_masks]
    ious = maskUtils.iou(enc_masks, enc_masks, [0]*len(enc_masks))
    
    return ious


def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def is_outlier(idx, ious, iou_th, n_th):
    return sum(np.where(ious[idx] > iou_th, 1, 0)) < n_th


def get_overlapping_indices(idx, ious, iou_th):
    return [i[0] for i in np.argwhere(ious[idx] > iou_th) if i[0] != idx]


def iou_sum(idx, ious, weights=None):
    if weights is not None:
        return np.sum(weights * ious[idx])
    return np.sum(ious[idx])


def ensemble_rles(rles, weights=None, low_iou_th=0.2, min_overlaps=3):
    """
    Combine predictions from multiple models by filtering outliers and less confident replicates.
    Two rles that overlap more than low_iou_th are replicates. 
    
    1.  Prediction outliers are removed. A pred with less than min_overlaps replicates is considered
    to be an outlier.
    
    2.  From replicates, only the pred that has the highest sum of overlapping ious is kept.
    
    Note that returned preds may contain overlapping rles (can only overlap less than low_iou_th)
    
    Argument:
        rles: list of rle string
        weights: optional weight for each rle - used for weighting IOU sums in rle copmarison
        low_iou_th: rles that overlap more than this are considered replicates
        min_overlaps: if rle has less than (min_overlaps - 1) replicates, it is discarded
    
    Returns:
        filtered_rles: list of rles that remain after filtering operations
    """
    ious = ious_of_rles(rles)
    
    # Check outliers
    delete_list = [False for _ in rles]
    probs = []
    for n in range(len(rles)):
        delete_list[n] = is_outlier(n, ious, iou_th=low_iou_th, n_th=min_overlaps)
        
    for n in range(len(rles)):
        replicate_idxs = get_overlapping_indices(n, ious, iou_th=low_iou_th)
        n_sum = iou_sum(n, ious, weights=weights)
        weight = 1. if weights is None else weights[n]
        probs.append(weight * n_sum)
        
        # compare against replicates and zero out if some of the reps is higher
        for idx in replicate_idxs:
            if iou_sum(idx, ious, weights=weights) > n_sum:
                delete_list[n] = True
                break
    
    rles = [rle for (rle, delete) in zip(rles, delete_list) if not delete]
    probs = [-1 * p for (p, delete) in zip(probs, delete_list) if not delete]
    
    # sort highest probs first
    probs, rles = zip(*sorted(zip(probs, rles)))
    
    return rles

def remove_overlap_in_rles(rles, shape=(520,704), min_size_left=0.7):
    """ Rles should have highest probs first """
    new_rles = []
    used = np.zeros(shape, dtype=np.int32)
    for rle in rles:
        binary_mask = rle_decode(rle, shape=(*shape,1)).astype(np.uint8)[:,:,0]
        area_before_overlap_remove = binary_mask.sum()
        binary_mask = binary_mask * (1-used)
        area_after_overlap_remove = binary_mask.sum()
        
        if (area_after_overlap_remove / (area_before_overlap_remove + 1e-6)) < min_size_left:
            continue
        used += binary_mask
        new_rles.append(rle_encode(binary_mask))
    return new_rles

CLASS_NAMES = ['cort', 'astro', 'shsy5y']

# Cell type class-specific parameters
CELLPOSE_DIAMETERS = [
    [20], # cort 
    [30], # astro
    [20],  # shsy5y
]

CELLPOSE_MASK_TH = [
    [0.1],  # cort
    [-0.4], # astro
    [0.2],   # shsy5y
]

# area less than this will get discarded - not used now
MIN_PIXEL_THS = [
    15, # cort
    15, # astro
    15  # shsy5y
]

CELLPOSE_FLOW_THS = [
    0.4, # cort
    0.5, # astro
    0.45 # shsy5y
] 

# Read classifier predictions for each id 
with open('id2class.json') as f:
    id2class = json.load(f)

test_dir = Path('../input/sartorius-cell-instance-segmentation/test')
test_files = [fname for fname in test_dir.iterdir()]

cellpose_models = [
    # cort
    [models.CellposeModel(
        gpu=True,
        pretrained_model=[
            # cyto-1 fold models
            "../input/sartoriuscellposemodels/cyto-1/cort/fold-0/ep499",
            "../input/sartoriuscellposemodels/cyto-1/cort/fold-1/ep499",
            "../input/sartoriuscellposemodels/cyto-1/cort/fold-2/ep499",
            "../input/sartoriuscellposemodels/cyto-1/cort/fold-3/ep499",
            "../input/sartoriuscellposemodels/cyto-1/cort/fold-4/ep499",
            
            # cyto-2 (only fold 4 was trained)
            "../input/sartoriuscellposemodels/cyto-2/cort/fold-4/ep299",
            
            # cyto-2-pseudo fold models
            "../input/sartoriuscellposemodels/cyto-2-pseudo/cort/fold-0/ep299",
            "../input/sartoriuscellposemodels/cyto-2-pseudo/cort/fold-1/ep299",
            "../input/sartoriuscellposemodels/cyto-2-pseudo/cort/fold-2/ep299",
            "../input/sartoriuscellposemodels/cyto-2-pseudo/cort/fold-3/ep299",
            "../input/sartoriuscellposemodels/cyto-2-pseudo/cort/fold-4/ep299",
            
            # pseudo-2
            "../input/sartoriuscellposemodels/cyto-2-pseudo-2/cort/fold-0/ep101",
            "../input/sartoriuscellposemodels/cyto-2-pseudo-2/cort/fold-1/ep101",
            "../input/sartoriuscellposemodels/cyto-2-pseudo-2/cort/fold-2/ep101",
            "../input/sartoriuscellposemodels/cyto-2-pseudo-2/cort/fold-3/ep101",
            "../input/sartoriuscellposemodels/cyto-2-pseudo-2/cort/fold-4/ep101",
    ])],
    # astro
    [models.CellposeModel(
        gpu=True,
        pretrained_model=[
            # cyto-1 fold models
            "../input/sartoriuscellposemodels/cyto-1/astro/fold-0/ep499",
            "../input/sartoriuscellposemodels/cyto-1/astro/fold-1/ep499",
            "../input/sartoriuscellposemodels/cyto-1/astro/fold-2/ep499",
            "../input/sartoriuscellposemodels/cyto-1/astro/fold-3/ep499",
            "../input/sartoriuscellposemodels/cyto-1/astro/fold-4/ep499",
            
            # cyto-2
            "../input/sartoriuscellposemodels/cyto-2/astro/fold-4/ep299",
            
            # cyto-2-pseudo
            "../input/sartoriuscellposemodels/cyto-2-pseudo/astro/fold-0/ep299",
            "../input/sartoriuscellposemodels/cyto-2-pseudo/astro/fold-1/ep299",
            "../input/sartoriuscellposemodels/cyto-2-pseudo/astro/fold-2/ep299",
            "../input/sartoriuscellposemodels/cyto-2-pseudo/astro/fold-3/ep299",
            "../input/sartoriuscellposemodels/cyto-2-pseudo/astro/fold-4/ep299",
            
            # pseudo-2
            "../input/sartoriuscellposemodels/cyto-2-pseudo-2/astro/fold-0/ep101",
            "../input/sartoriuscellposemodels/cyto-2-pseudo-2/astro/fold-1/ep101",
            "../input/sartoriuscellposemodels/cyto-2-pseudo-2/astro/fold-2/ep101",
            "../input/sartoriuscellposemodels/cyto-2-pseudo-2/astro/fold-3/ep101",
            "../input/sartoriuscellposemodels/cyto-2-pseudo-2/astro/fold-4/ep101",
            
    ])],
    # shsy5y
    [models.CellposeModel(
        gpu=True,
        pretrained_model=[
            # cyto-1
            "../input/sartoriuscellposemodels/cyto-1/shsy5y/fold-0/ep159",
            "../input/sartoriuscellposemodels/cyto-1/shsy5y/fold-1/ep159",
            "../input/sartoriuscellposemodels/cyto-1/shsy5y/fold-2/ep159",
            "../input/sartoriuscellposemodels/cyto-1/shsy5y/fold-3/ep159",
            "../input/sartoriuscellposemodels/cyto-1/shsy5y/fold-4/ep159",
            
            # cyto-2
            "../input/sartoriuscellposemodels/cyto-2/shsy5y/fold-4/ep159",

            # cyto-2-pseudo
            "../input/sartoriuscellposemodels/cyto-2-pseudo/shsy5y/fold-0/ep159",
            "../input/sartoriuscellposemodels/cyto-2-pseudo/shsy5y/fold-1/ep159",
            "../input/sartoriuscellposemodels/cyto-2-pseudo/shsy5y/fold-2/ep159",
            "../input/sartoriuscellposemodels/cyto-2-pseudo/shsy5y/fold-3/ep159",
            "../input/sartoriuscellposemodels/cyto-2-pseudo/shsy5y/fold-4/ep159",
            
            # pseudo-2
            "../input/sartoriuscellposemodels/cyto-2-pseudo-2/shsy5y/fold-0/ep121",
            "../input/sartoriuscellposemodels/cyto-2-pseudo-2/shsy5y/fold-1/ep121",
            "../input/sartoriuscellposemodels/cyto-2-pseudo-2/shsy5y/fold-2/ep121",
            "../input/sartoriuscellposemodels/cyto-2-pseudo-2/shsy5y/fold-3/ep121",
            "../input/sartoriuscellposemodels/cyto-2-pseudo-2/shsy5y/fold-4/ep121",
    ])]
]

ids, masks = [],[]
for fn in test_files:
    
    # get class index
    fn_basename = os.path.basename(fn).replace('.png','').replace('.tif','').replace('.tiff','')
    _cl = id2class[fn_basename]
    print(f'{fn_basename} - class {CLASS_NAMES[_cl]}')
    flow_th = CELLPOSE_FLOW_THS[_cl]
    
    rle_list = []
    for i, cp_model in enumerate(cellpose_models[_cl]):
        
        # Normal image
        preds, flows, _ = cp_model.eval(
            io.imread(str(fn)), 
            diameter=CELLPOSE_DIAMETERS[_cl][i],
            flow_threshold=flow_th,
            channels=[0,0],
            augment=True,
            mask_threshold=CELLPOSE_MASK_TH[_cl][i],
            resample=True)
        print(f'CP model {i} predicted {preds.max()} cells.')
        for j in range (1, preds.max() + 1):
            rle_list.append(rle_encode(preds == j))
            
        # HFlipped image
        preds, flows, _ = cp_model.eval(
            cv2.flip(io.imread(str(fn)), 0), 
            diameter=CELLPOSE_DIAMETERS[_cl][i], 
            flow_threshold=flow_th,
            channels=[0,0],
            augment=True,
            mask_threshold=CELLPOSE_MASK_TH[_cl][i],
            resample=True)
        preds = cv2.flip(preds, 0)
        
        print(f'HFlip - CP model {i} predicted {preds.max()} cells.')
        for j in range (1, preds.max() + 1):
            rle_list.append(rle_encode(preds == j))
            
        # Transposed image
        preds, flows, _ = cp_model.eval(
            np.transpose(io.imread(str(fn)), axes=(1,0)), 
            diameter=CELLPOSE_DIAMETERS[_cl][i],
            flow_threshold=flow_th,
            channels=[0,0],
            augment=True,
            mask_threshold=CELLPOSE_MASK_TH[_cl][i],
            resample=True)
        preds = np.transpose(preds, axes=(1,0))
        print(f'Transposed - CP model {i} predicted {preds.max()} cells.')
        for j in range (1, preds.max() + 1):
            rle_list.append(rle_encode(preds == j))
            
        # Transposed HFlipped image
        preds, flows, _ = cp_model.eval(
            cv2.flip(np.transpose(io.imread(str(fn)), axes=(1,0)), 0), 
            diameter=CELLPOSE_DIAMETERS[_cl][i],
            flow_threshold=flow_th,
            channels=[0,0],
            augment=True,
            mask_threshold=CELLPOSE_MASK_TH[_cl][i],
            resample=True)
        preds = cv2.flip(np.transpose(preds, axes=(1,0)), 0)
        print(f'Transposed + HFlip - CP model {i} predicted {preds.max()} cells.')
        for j in range (1, preds.max() + 1):
            rle_list.append(rle_encode(preds == j))
            
    
    # 4xTTA ensemble
    # allow only preds that have min_overlaps replicates
    filtered_rles = ensemble_rles(rle_list, low_iou_th=0.5, min_overlaps=3)
    filtered_rles = remove_overlap_in_rles(filtered_rles, min_size_left=0.5)
    print(f'NMS left {len(filtered_rles)} cells.')
    #filtered_rles = rle_list
    
    for rle in filtered_rles:
        ids.append(fn.stem)
        masks.append(rle)
        
pd.DataFrame({'id':ids, 'predicted':masks}).to_csv('submission.csv', index=False)

In [ ]:
!python cellpose_inference.py

## Detectron2 Patch

In [ ]:
def paste_masks_in_image(masks, boxes, image_shape, threshold=0.5):

    assert masks.shape[-1] == masks.shape[-2], "Only square mask predictions are supported"
    N = len(masks)
    if N == 0:
        return masks.new_empty((0,) + image_shape, dtype=torch.uint8)
    if not isinstance(boxes, torch.Tensor):
        boxes = boxes.tensor
    device = boxes.device
    assert len(boxes) == N, boxes.shape

    img_h, img_w = image_shape

    # The actual implementation split the input into chunks,
    # and paste them chunk by chunk.
    if device.type == "cpu":
        # CPU is most efficient when they are pasted one by one with skip_empty=True
        # so that it performs minimal number of operations.
        num_chunks = N
    else:
        # GPU benefits from parallelism for larger chunks, but may have memory issue
        num_chunks = int(np.ceil(N * img_h * img_w * BYTES_PER_FLOAT / GPU_MEM_LIMIT))
        assert (
            num_chunks <= N
        ), "Default GPU_MEM_LIMIT in mask_ops.py is too small; try increasing it"
    chunks = torch.chunk(torch.arange(N, device=device), num_chunks)

    img_masks = torch.zeros(
        N, img_h, img_w, device=device, dtype=torch.float32
    )
    for inds in chunks:
        masks_chunk, spatial_inds = _do_paste_mask(
            masks[inds, None, :, :], boxes[inds], img_h, img_w, skip_empty=device.type == "cpu"
        )
        img_masks[(inds,) + spatial_inds] = masks_chunk

    return img_masks


def BitMasks__init__(self, tensor: Union[torch.Tensor, np.ndarray]):

    device = tensor.device if isinstance(tensor, torch.Tensor) else torch.device("cpu")
    tensor = torch.as_tensor(tensor, dtype=torch.float32, device=device)
    assert tensor.dim() == 3, tensor.size()
    self.image_size = tensor.shape[1:]
    self.tensor = tensor


PATCH = False

if PATCH:
    detectron2.layers.mask_ops.paste_masks_in_image.__code__ = paste_masks_in_image.__code__
    print('detectron2.layers.mask_ops.paste_masks_in_image is patched.')
    detectron2.structures.masks.BitMasks.__init__.__code__ = BitMasks__init__.__code__
    print('detectron2.structures.masks.BitMasks.init is patched.')


## Detectron2 Models

In [ ]:
def load_detectron2_models(model_directory, folds_to_use):

    """
    Load detectron models from the given directory
    
    Parameters
    ----------
    model_directory (str): Directory of models, trainer_config and detectron_config
    folds_to_use (list): List of folds to load
    
    Returns
    -------
    models (dict): Dictionary of models
    """

    print(f'\nLoading Detectron2 models from {model_directory}')
    models = {}
    model_names = sorted(glob(f'{model_directory}/*.pth'))
    trainer_config = yaml.load(open(f'{model_directory}/trainer_config.yaml', 'r'), Loader=yaml.FullLoader)

    for fold, weights_path in enumerate(model_names, start=1):

        if fold in folds_to_use:

            detectron_config = get_cfg()
            detectron_config.merge_from_file(model_zoo.get_config_file(trainer_config['MODEL']['model_zoo_path']))
            detectron_config.MODEL.WEIGHTS = weights_path
            detectron_config.merge_from_file(f'{model_directory}/detectron_config.yaml')

            # Disable NMS and score thresholds so it can be done class-wise
            detectron_config.MODEL.ROI_HEADS.NMS_THRESH_TEST = 1.0
            detectron_config.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0
            detectron_config.TEST.DETECTIONS_PER_IMAGE = 1000

            model = DefaultPredictor(detectron_config)
            models[fold] = model
            print(f'Loaded model {weights_path} into memory')

    return models, detectron_config

In [ ]:
LOAD_MASK_RCNN2_MODELS = True
if LOAD_MASK_RCNN2_MODELS:
    detectron2_mask_rcnn_models2, detectron2_config2 = load_detectron2_models(
        f'../input/sartorius-cell-instance-segmentation-dataset/detectron2_mask_rcnn2',
        folds_to_use=[1, 2, 3, 4, 5, 6]
    )
    
LOAD_MASK_RCNN3_MODELS = False
if LOAD_MASK_RCNN3_MODELS:
    detectron2_mask_rcnn_models3, detectron2_config3 = load_detectron2_models(
        f'../input/sartorius-cell-instance-segmentation-dataset/detectron2_mask_rcnn3',
        folds_to_use=[1, 2, 3, 4, 5, 6]
    )

LOAD_MASK_RCNN4_MODELS = False
if LOAD_MASK_RCNN4_MODELS:
    detectron2_mask_rcnn_models4, detectron2_config4 = load_detectron2_models(
        f'../input/sartorius-cell-instance-segmentation-dataset/detectron2_mask_rcnn4',
        folds_to_use=[1, 2, 3, 4, 5, 6]
    )

## Post-processing

In [ ]:
def prepare_boxes(boxes, scores, labels, masks=None):
    result_boxes = boxes.copy()

    cond = (result_boxes < 0)
    cond_sum = cond.astype(np.int32).sum()
    if cond_sum > 0:
        print('Warning. Fixed {} boxes coordinates < 0'.format(cond_sum))
        result_boxes[cond] = 0

    cond = (result_boxes > 1)
    cond_sum = cond.astype(np.int32).sum()
    if cond_sum > 0:
        print('Warning. Fixed {} boxes coordinates > 1. Check that your boxes was normalized at [0, 1]'.format(cond_sum))
        result_boxes[cond] = 1

    boxes1 = result_boxes.copy()
    result_boxes[:, 0] = np.min(boxes1[:, [0, 2]], axis=1)
    result_boxes[:, 2] = np.max(boxes1[:, [0, 2]], axis=1)
    result_boxes[:, 1] = np.min(boxes1[:, [1, 3]], axis=1)
    result_boxes[:, 3] = np.max(boxes1[:, [1, 3]], axis=1)

    area = (result_boxes[:, 2] - result_boxes[:, 0]) * (result_boxes[:, 3] - result_boxes[:, 1])
    cond = (area == 0)
    cond_sum = cond.astype(np.int32).sum()
    if cond_sum > 0:
        print('Warning. Removed {} boxes with zero area!'.format(cond_sum))
        result_boxes = result_boxes[area > 0]
        scores = scores[area > 0]
        labels = labels[area > 0]
        if masks is not None:
            masks = masks[area > 0]

    return result_boxes, scores, labels, masks


def cpu_soft_nms_float(dets, sc, Nt, sigma, thresh, method):

    # indexes concatenate boxes with the last column
    N = dets.shape[0]
    indexes = np.array([np.arange(N)])
    dets = np.concatenate((dets, indexes.T), axis=1)

    # the order of boxes coordinate is [y1, x1, y2, x2]
    y1 = dets[:, 1]
    x1 = dets[:, 0]
    y2 = dets[:, 3]
    x2 = dets[:, 2]
    scores = sc
    areas = (x2 - x1) * (y2 - y1)

    for i in range(N):
        # intermediate parameters for later parameters exchange
        tBD = dets[i, :].copy()
        tscore = scores[i].copy()
        tarea = areas[i].copy()
        pos = i + 1

        #
        if i != N - 1:
            maxscore = np.max(scores[pos:], axis=0)
            maxpos = np.argmax(scores[pos:], axis=0)
        else:
            maxscore = scores[-1]
            maxpos = 0
        if tscore < maxscore:
            dets[i, :] = dets[maxpos + i + 1, :]
            dets[maxpos + i + 1, :] = tBD
            tBD = dets[i, :]

            scores[i] = scores[maxpos + i + 1]
            scores[maxpos + i + 1] = tscore
            tscore = scores[i]

            areas[i] = areas[maxpos + i + 1]
            areas[maxpos + i + 1] = tarea
            tarea = areas[i]

        # IoU calculate
        xx1 = np.maximum(dets[i, 1], dets[pos:, 1])
        yy1 = np.maximum(dets[i, 0], dets[pos:, 0])
        xx2 = np.minimum(dets[i, 3], dets[pos:, 3])
        yy2 = np.minimum(dets[i, 2], dets[pos:, 2])

        w = np.maximum(0.0, xx2 - xx1)
        h = np.maximum(0.0, yy2 - yy1)
        inter = w * h
        ovr = inter / (areas[i] + areas[pos:] - inter)

        # Three methods: 1.linear 2.gaussian 3.original NMS
        if method == 1:  # linear
            weight = np.ones(ovr.shape)
            weight[ovr > Nt] = weight[ovr > Nt] - ovr[ovr > Nt]
        elif method == 2:  # gaussian
            weight = np.exp(-(ovr * ovr) / sigma)
        else:  # original NMS
            weight = np.ones(ovr.shape)
            weight[ovr > Nt] = 0

        scores[pos:] = weight * scores[pos:]

    # select the boxes and keep the corresponding indexes
    inds = dets[:, 4][scores > thresh]
    keep = inds.astype(int)
    return keep


@jit(nopython=True)
def nms_float_fast(dets, scores, thresh):

    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]

    areas = (x2 - x1) * (y2 - y1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1)
        h = np.maximum(0.0, yy2 - yy1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)
        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]

    return keep


def nms_method(boxes, scores, labels, masks=None, method=3, iou_thr=0.5, sigma=0.5, thresh=0.001, weights=None):

    # If weights are specified
    if weights is not None:
        if len(boxes) != len(weights):
            print('Incorrect number of weights: {}. Must be: {}. Skip it'.format(len(weights), len(boxes)))
        else:
            weights = np.array(weights)
            for i in range(len(weights)):
                scores[i] = (np.array(scores[i]) * weights[i]) / weights.sum()

    # We concatenate everything
    boxes = np.concatenate(boxes)
    scores = np.concatenate(scores)
    labels = np.concatenate(labels)
    if masks is not None:
        masks = np.concatenate(masks)

    # Fix coordinates and removed zero area boxes
    boxes, scores, labels, masks = prepare_boxes(boxes, scores, labels, masks)

    # Run NMS independently for each label
    unique_labels = np.unique(labels)
    final_boxes = []
    final_scores = []
    final_labels = []
    if masks is not None:
        final_masks = []

    for l in unique_labels:
        condition = (labels == l)
        boxes_by_label = boxes[condition]
        scores_by_label = scores[condition]
        labels_by_label = np.array([l] * len(boxes_by_label))
        if masks is not None:
            masks_by_label = masks[condition]

        if method != 3:
            keep = cpu_soft_nms_float(boxes_by_label.copy(), scores_by_label.copy(), Nt=iou_thr, sigma=sigma, thresh=thresh, method=method)
        else:
            # Use faster function
            keep = nms_float_fast(boxes_by_label, scores_by_label, thresh=iou_thr)

        final_boxes.append(boxes_by_label[keep])
        final_scores.append(scores_by_label[keep])
        final_labels.append(labels_by_label[keep])
        if masks is not None:
            final_masks.append(masks_by_label[keep])
    final_boxes = np.concatenate(final_boxes)
    final_scores = np.concatenate(final_scores)
    final_labels = np.concatenate(final_labels)
    if masks is not None:
        final_masks = np.concatenate(final_masks)

    return final_boxes, final_scores, final_labels, final_masks


def nms(boxes, scores, labels, masks=None, iou_thr=0.5, weights=None):

    return nms_method(boxes, scores, labels, masks, method=3, iou_thr=iou_thr, weights=weights)


def soft_nms(boxes, scores, labels, masks=None, method=2, iou_thr=0.5, sigma=0.5, thresh=0.001, weights=None):

    return nms_method(boxes, scores, labels, masks, method=method, iou_thr=iou_thr, sigma=sigma, thresh=thresh, weights=weights)


In [ ]:
def fix_overlaps(masks, area_threshold, mask_area_order='descending'):

    """
    Remove overlapping regions of the given masks
    
    Parameters
    ----------
    masks [numpy.ndarray of shape (n_objects, height, width)]: 2d binary masks
    area_threshold (int): Threshold for dropping small islands after removing overlapping regions
    mask_area_order (str): Whether to sort masks by their area in descending or ascending order
    
    Returns
    -------
    non_overlapping_masks [numpy.ndarray of shape (n_objects, height, width)]: 2d binary masks with no overlapping regions
    """
    
    # Sort masks by their areas in descending or ascending order
    # This will give importance to larger or smaller masks
    mask_areas = np.sum(masks, axis=(1, 2))
    mask_areas = mask_areas.astype(np.uint16)
    if mask_area_order == 'descending':
        masks = masks[np.argsort(mask_areas)[::-1], :, :]
    else:
        masks = masks[np.argsort(mask_areas), :, :]

    non_overlapping_masks = []
    used_pixels = np.zeros(masks.shape[1:], dtype=np.uint16)

    for mask in masks:
        mask = mask * (1 - used_pixels)
        # Filter out objects smaller than area_threshold after removing overlapping regions
        if np.sum(mask) >= area_threshold:
            used_pixels += mask
            non_overlapping_masks.append(mask)
    
    non_overlapping_masks = np.stack(non_overlapping_masks).astype(bool)
    return non_overlapping_masks


def filter_predictions(predictions, box_height_scale, box_width_scale, iou_threshold=None, nms_weights=None, score_threshold=None, verbose=False):

    """
    Filter predictions with NMS and scores
    
    Parameters
    ----------
    prediction (list): List of one or multiple dictionaries of predicted boxes, labels, scores and masks as numpy arrays
    box_height_scale (int): Height of the image
    box_width_scale (int): Width of the image
    iou_threshold (float): Supress boxes and masks with NMS with this threshold (0 <= iou_threshold <= 1)
    nms_weights (list): List of weights of predictions (nms_weights must have same length with predictions)
    score_threshold (float): Remove boxes and masks based on their scores with this threshold (0 <= score_threshold <= 1)
    verbose (str): Verbosity flag
    
    Returns
    -------
    prediction (dict): Dictionary of predicted boxes, labels, scores and masks as numpy arrays
    """

    boxes_list = []
    scores_list = []
    labels_list = []
    masks_list = []

    # Storing predictions of multiple models into lists
    for prediction in predictions:
        # Scale box coordinates between 0 and 1
        prediction['boxes'][:, 0] /= box_width_scale
        prediction['boxes'][:, 1] /= box_height_scale
        prediction['boxes'][:, 2] /= box_width_scale
        prediction['boxes'][:, 3] /= box_height_scale

        boxes_list.append(prediction['boxes'].tolist())
        scores_list.append(prediction['scores'].tolist())
        labels_list.append(prediction['labels'].tolist())
        masks_list.append(prediction['masks'])

        if verbose:
            print(f'{len(prediction["scores"])} objects are predicted with {np.mean(prediction["scores"]):.4f} average score')
            
    del predictions

    # Supress overlapping boxes with NMS
    boxes, scores, labels, masks = nms(
        boxes=boxes_list,
        scores=scores_list,
        labels=labels_list,
        masks=masks_list,
        iou_thr=iou_threshold,
        weights=nms_weights
    )
    
    del boxes_list, scores_list, labels_list, masks_list
    if verbose:
        print(f'{len(scores)} objects are kept after applying {iou_threshold} nms iou threshold with {np.mean(scores):.4f} average score')

    # Rescale box coordinates between image height and width
    boxes[:, 0] *= box_width_scale
    boxes[:, 1] *= box_height_scale
    boxes[:, 2] *= box_width_scale
    boxes[:, 3] *= box_height_scale

    # Filter out boxes based on scores
    score_condition = scores >= score_threshold
    boxes = boxes[score_condition]
    scores = scores[score_condition]
    masks = masks[score_condition]
    labels = labels[score_condition]

    if verbose:
        print(f'{len(scores)} objects are kept after applying {score_threshold} score threshold with {np.mean(scores):.4f} average score')

    return boxes, scores, labels, masks


def get_iou_matrix_from_boxes(bounding_boxes1, bounding_boxes2):

    """
    Calculate IoU matrix between two sets of bounding boxes
    
    Parameters
    ----------
    bounding_boxes1 [numpy.ndarray of shape (n_objects, 4)]: Bounding boxes
    bounding_boxes2 [numpy.ndarray of shape (m_objects, 4)]: Bounding boxes
    
    Returns
    -------
    iou_matrix [numpy.ndarray of shape (n_objects, m_objects)]: IoU matrix between two sets of bounding boxes
    """

    bounding_boxes1_x1, bounding_boxes1_y1, bounding_boxes1_x2, bounding_boxes1_y2 = np.split(bounding_boxes1, 4, axis=1)
    bounding_boxes2_x1, bounding_boxes2_y1, bounding_boxes2_x2, bounding_boxes2_y2 = np.split(bounding_boxes2, 4, axis=1)

    xa = np.maximum(bounding_boxes1_x1, np.transpose(bounding_boxes2_x1))
    ya = np.maximum(bounding_boxes1_y1, np.transpose(bounding_boxes2_y1))
    xb = np.minimum(bounding_boxes1_x2, np.transpose(bounding_boxes2_x2))
    yb = np.minimum(bounding_boxes1_y2, np.transpose(bounding_boxes2_y2))

    inter_area = np.maximum((xb - xa + 1), 0) * np.maximum((yb - ya + 1), 0)
    box_a_area = (bounding_boxes1_x2 - bounding_boxes1_x1 + 1) * (bounding_boxes1_y2 - bounding_boxes1_y1 + 1)
    box_b_area = (bounding_boxes2_x2 - bounding_boxes2_x1 + 1) * (bounding_boxes2_y2 - bounding_boxes2_y1 + 1)
    iou_matrix = inter_area / (box_a_area + np.transpose(box_b_area) - inter_area)

    return iou_matrix


def get_iou_matrix_from_masks(masks1, masks2):
    
    """
    Calculate IOU matrix between two sets of masks
    
    Parameters
    ----------
    masks1 [numpy.ndarray of shape (n_objects, height, width)]: 2d binary masks
    masks2 [numpy.ndarray of shape (m_objects, height, width)]: 2d binary masks
    
    Returns
    -------
    iou_matrix [numpy.ndarray of shape (n_objects, m_objects)]: IoU matrix between two sets of masks
    """
    
    if len(list(masks1)) == 0 or len(list(masks2)) == 0:
        print(f'empty predictions - masks1 len {len(list(masks1))}, masks2 len {len(list(masks2))}')
        return np.array([[]])
    
    enc_masks1 = [mask_util.encode(np.asarray(p, order='F')) for p in (masks1 > 0.5).astype(np.uint8)]
    enc_masks2 = [mask_util.encode(np.asarray(p, order='F')) for p in (masks2 > 0.5).astype(np.uint8)]
    iou_matrix = mask_util.iou(enc_masks1, enc_masks2, [0] * len(enc_masks1))
    
    return iou_matrix



def blend_masks(prediction_boxes, prediction_masks, iou_threshold=0.9, label_threshold=0.5, iou_method='boxes', drop_single_components=True):

    """
    Blend prediction masks of multiple models based on IoU
    
    Parameters
    ----------
    prediction_boxes [list of shape (n_models)]: Bounding box predictions of multiple models
    prediction_masks [list of shape (n_models)]: Mask predictions of multiple models
    iou_threshold (int): IoU threshold for blending masks (0 <= iou_threshold <= 1)
    label_threshold (int): Label threshold for converting soft predictions to labels (0 <= iou_threshold <= 1)
    drop_single_components (bool): Whether to discard predictions without connections or not
    
    Returns
    -------
    blended_masks [numpy.ndarray of shape (n_objects, height, width)]: Blended binary masks
    """

    iou_matrices = {}

    # Create all combinations of IoU matrices from given predictions
    for i in range(len(prediction_masks)):
        for j in range(i, len(prediction_masks)):
            if i == j:
                continue
            
            if iou_method == 'boxes':
                iou_matrix = get_iou_matrix_from_boxes(prediction_boxes[i], prediction_boxes[j])
            elif iou_method == 'masks':
                iou_matrix = get_iou_matrix_from_masks(prediction_masks[i], prediction_masks[j])
            
            iou_matrices[f'{i + 1}_{j + 1}'] = iou_matrix

    # Create a graph to store connected bounding boxes
    bounding_box_graph = nx.Graph()

    # Add all masks from all models as nodes
    for model_idx, boxes in enumerate(prediction_masks, start=1):
        nodes = [f'model{model_idx}_box{box_idx}' for box_idx in np.arange(len(boxes))]
        bounding_box_graph.add_nodes_from(nodes)
        
    del prediction_boxes

    # Add edges between nodes with IoU >= iou_threshold
    for model_combination, iou_matrix in iou_matrices.items():
        matching_boxes_idx = np.where(iou_matrix >= iou_threshold)
        model1_idx, model2_idx = model_combination.split('_')
        edges = [(f'model{model1_idx}_box{box1}', f'model{model2_idx}_box{box2}') for box1, box2 in zip(*matching_boxes_idx)]
        bounding_box_graph.add_edges_from(edges)

    del iou_matrices
    blended_masks = []

    for connections in nx.connected_components(bounding_box_graph):
        if len(connections) == 1:
            # Skip mask if its bounding isn't connected to any other bounding box
            if drop_single_components:
                continue
            else:
                # Append mask directly if its bounding box isn't connected to any other bounding box
                model_idx, box_idx = list(connections)[0].split('_')
                model_idx = int(model_idx.replace('model', ''))
                box_idx = int(box_idx.replace('box', ''))
                blended_masks.append(prediction_masks[model_idx - 1][box_idx])
        else:
            # Blend mask with its connections and append
            blended_mask = np.zeros((520, 704), dtype=np.float32)
            for connection in connections:
                model_idx, box_idx = connection.split('_')
                model_idx = int(model_idx.replace('model', ''))
                box_idx = int(box_idx.replace('box', ''))
                # Divide soft predictions with number of connections and accumulate on blended_mask
                blended_mask += (prediction_masks[model_idx - 1][box_idx] / len(connections))
            blended_masks.append(blended_mask)
            
    del prediction_masks, bounding_box_graph
    blended_masks = np.stack(blended_masks)
    # Convert soft predictions to binary labels
    blended_masks = np.uint8(blended_masks >= label_threshold)

    return blended_masks

## Detectron2 Inference

In [ ]:
def predict_single_image(image, model):

    """
    Predict given image with given model and move predictions to cpu
    
    Parameters
    ----------
    image [numpy.ndarray of shape (height, width, channel)]: Image (BGR)
    model (torch.nn.Module): Detectron2 Model
    
    Returns
    -------
    prediction (dict): Dictionary of predicted boxes, labels, scores and masks as numpy arrays
    """

    prediction = model(image)
    prediction = {
        'boxes': prediction['instances'].pred_boxes.tensor.cpu().numpy(),
        'labels': prediction['instances'].pred_classes.cpu().numpy(),
        'scores': prediction['instances'].scores.cpu().numpy(),
        'masks': prediction['instances'].pred_masks.cpu().numpy()
    }

    return prediction

In [ ]:
detectron2_mask_rcnn_post_processing_parameters = {
    'nms_iou_thresholds': {
        0: 0.3,
        1: 0.3,
        2: 0.3
    },
    'score_thresholds': {
        0: 0.8,
        1: 0.45,
        2: 0.45
    },
    'area_thresholds': {
        0: 60,
        1: 60,
        2: 150
    },
    'blend_iou_thresholds': {
        0: 0.8,
        1: 0.8,
        2: 0.8
    },
    'pixel_label_thresholds': {
        0: 0.5,
        1: 0.5,
        2: 0.5
    }
}

test_cell_types = []


def detectron2_inference(test_images, models, post_processing_parameters):
    
    df = pd.DataFrame(columns=['id', 'predicted'])
    
    for file_name in tqdm(test_images):
        
        all_prediction_boxes = []
        all_prediction_masks = []
        cell_types = []
        
        image = cv2.imread(f'../input/sartorius-cell-instance-segmentation/test/{file_name}')
        for fold, model in models.items():
            prediction = predict_single_image(image=image, model=model)
            # Select cell type as the most predicted label
            cell_type = mode(prediction['labels'])[0][0]
            prediction_boxes, prediction_scores, prediction_labels, prediction_masks = filter_predictions(
                predictions=[prediction],
                box_height_scale=image.shape[0],
                box_width_scale=image.shape[1],
                iou_threshold=post_processing_parameters['nms_iou_thresholds'][cell_type],
                nms_weights=None,
                score_threshold=post_processing_parameters['score_thresholds'][cell_type],
                verbose=False
            )

            all_prediction_boxes.append(prediction_boxes)
            all_prediction_masks.append(prediction_masks)
            cell_types.append(cell_type)
                
        cell_type = mode(cell_types)[0][0]
        test_cell_types.append(cell_type)
        
        # Blend prediction masks of multiple models based on IoU 
        blended_prediction_masks = blend_masks(
            prediction_boxes=all_prediction_boxes,
            prediction_masks=all_prediction_masks,
            iou_threshold=post_processing_parameters['blend_iou_thresholds'][cell_type],
            iou_method='masks',
            label_threshold=post_processing_parameters['pixel_label_thresholds'][cell_type],
            drop_single_components=True
        )
        
        for prediction_mask in blended_prediction_masks:
            rle_encoded_mask = encode_rle_mask(prediction_mask)
            df = df.append({'id': file_name.split('.')[0], 'predicted': rle_encoded_mask}, ignore_index=True)
                
    return df


In [ ]:
test_images = os.listdir('../input/sartorius-cell-instance-segmentation/test')

df_mask_rcnn2_predictions = detectron2_inference(
    test_images=test_images,
    models=detectron2_mask_rcnn_models2,
    post_processing_parameters=detectron2_mask_rcnn_post_processing_parameters
)
print(f'Mask R-CNN 2 - {df_mask_rcnn2_predictions.shape[0]} objects are predicted on {df_mask_rcnn2_predictions["id"].nunique()} images')
del detectron2_mask_rcnn_models2

df_cellpose_predictions = pd.read_csv('submission.csv')
print(f'Cellpose - {df_cellpose_predictions.shape[0]} objects are predicted on {df_cellpose_predictions["id"].nunique()} images')

In [ ]:
df_submission = pd.DataFrame(columns=['id', 'predicted'])

for idx, file_name in enumerate(test_images):
    
    image = cv2.imread(f'../input/sartorius-cell-instance-segmentation/test/{file_name}')
    
    mask_rcnn2_masks = df_mask_rcnn2_predictions.loc[df_mask_rcnn2_predictions['id'] == file_name.split('.')[0], 'predicted']
    mask_rcnn2_masks = np.stack([decode_rle_mask(rle_mask, shape=(image.shape[:2])) for rle_mask in mask_rcnn2_masks])
    mask_rcnn2_boxes = np.stack([mask_to_bounding_box(mask) for mask in mask_rcnn2_masks])
    
    cellpose_masks = df_cellpose_predictions.loc[df_cellpose_predictions['id'] == file_name.split('.')[0], 'predicted']
    cellpose_masks = np.stack([decode_rle_mask(rle_mask, shape=image.shape[:2]) for rle_mask in cellpose_masks])
    cellpose_boxes = np.stack([mask_to_bounding_box(mask) for mask in cellpose_masks])
    
    blended_masks = blend_masks(
        prediction_boxes=[mask_rcnn2_boxes, cellpose_boxes],
        prediction_masks=[mask_rcnn2_masks, cellpose_masks],
        iou_threshold=0.8,
        iou_method='masks',
        label_threshold=0.5,
        drop_single_components=False
    )
    
    blended_masks = fix_overlaps(
        blended_masks,
        area_threshold=detectron2_mask_rcnn_post_processing_parameters['area_thresholds'][test_cell_types[idx]],
        mask_area_order='ascending'
    )
    
    for mask in blended_masks:
        rle_encoded_mask = encode_rle_mask(mask)
        df_submission = df_submission.append({'id': file_name.split('.')[0], 'predicted': rle_encoded_mask}, ignore_index=True)


In [ ]:
df_submission.to_csv('submission.csv', index=False)
df_submission.head(15)